In [1]:
import time
import logging
import json
from pathlib import Path
from collections import defaultdict
from typing import List
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F

In [2]:
from pyspark.sql.types import *
import types
import ibm_boto3
from botocore.client import Config
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_utc_timestamp
import base64,getpass

In [3]:
conf = spark.sparkContext.getConf()
spark.stop()

# Parameters to be set

In [4]:
username = "hkaug_user01"  ## Need to add, should be hkaug_user0x 
password = "passw0rd123" ## No need to change 
wxd_username = f"ibmlhapikey_{username}" ## No need to change 
wxd_apikey = "gkOVBU7Pjv9P2hOyloVTxDb2pyfcjTgpfyjmO0Lq" ## Need to add, the apikey you generate at lab0 
b = base64.b64encode(bytes(f"{username}:{wxd_apikey}\n", 'utf-8')) ## No need to change 
base64_key = b.decode('utf-8') ## No need to change 
zen_api_key = f"ZenApiKey {base64_key}" ## No need to change 

# basic encoded creds
string_to_encode = username + ":" + password ## No need to change 
wxd_encoded_apikey="Basic "+ base64.b64encode(string_to_encode.encode("utf-8")).decode("utf-8") ## No need to change 

bucket_name = "incubation-workshops-bucket" ## No need to change 
bucket_ep = "https://s3.au-syd.cloud-object-storage.appdomain.cloud" ## No need to change 
bucket_access_key = "35fbec88f9eb4e6c9cc04491600dbfa4" ## No need to change 
bucket_secret_key = "8dc87c3e92806e3404b7a4289e9461d86bd0fde7cad020e4" ## No need to change 
catalog_name = "hk_aug_catalog0" ## No need to change 
schema_name = "hk_aug_schema0" ## No need to change 

transactions_silver_table = "xy1_transactions_silver" ## Need to change, the silver transactions table you just created  
cards_silver_table = "xy1_creditcards_silver" ## Need to change, the silver credit cards table you just created
customers_silver_table = "xy1_customers_silver" ## Need to change, the silver customers table you just created

gold_risk_table = "xy1_customers_risk_360" ## Need to change, the gold customer risk table name, should be [your name]_customers_risk_360
gold_campaign_table = "xy1_customers_campaign_360" ## Need to change, the gold customer campaign table name, should be [your name]_customers_campaign_360

mds_truststore_pass = "changeit" ## No need to change 

# Connection test

In [6]:
conf.setAll([
    ("spark.sql.catalogImplementation", "hive"), \
    ("spark.hive.metastore.uris", "thrift://ibm-lh-lakehouse-mds-thrift-svc.cpd-instance.svc.cluster.local:8380"), \
    ("spark.hive.metastore.use.SSL", "true"), \
    ("spark.hive.metastore.truststore.path", "file:///opt/ibm/jdk/lib/security/cacerts"), \
	("spark.hive.metastore.truststore.password", mds_truststore_pass), \
    ("spark.hive.metastore.truststore.type", "JKS"), \
    ("spark.hive.metastore.client.auth.mode", "PLAIN"), \
    ("spark.hive.metastore.client.plain.username", username), \
    ("spark.hive.metastore.client.plain.password", password), \
    ("spark.hadoop.wxd.apiKey", wxd_encoded_apikey), \
    (f"spark.hadoop.fs.s3a.bucket.{bucket_name}.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"), \
    (f"spark.hadoop.fs.s3a.bucket.{bucket_name}.endpoint", bucket_ep), \
    (f"spark.hadoop.fs.s3a.bucket.{bucket_name}.access.key", bucket_access_key), \
    (f"spark.hadoop.fs.s3a.bucket.{bucket_name}.secret.key", bucket_secret_key), \
    (f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog"), \
    (f"spark.sql.catalog.{catalog_name}.type" ,"hive"), \
    (f"spark.hadoop.fs.s3a.bucket.{bucket_name}.path.style.access", "true"), \
    ("spark.hadoop.fs.s3a.bucket.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"), \
    ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") 
])


In [7]:
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
print("spark instance inited")

spark instance inited


In [8]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='TBD'),
 Database(name='wxd_system_data_diag_presto645', catalog='spark_catalog', description='', locationUri='s3a://wxd-system/wxd_system_data_diag_presto645'),
 Database(name='test', catalog='spark_catalog', description='', locationUri='s3a://iceberg-bucket/test'),
 Database(name='hk_aug_schema', catalog='spark_catalog', description='', locationUri='s3a://incubation-workshops-bucket/hk_aug_schema'),
 Database(name='hk_aug_schema0', catalog='spark_catalog', description='', locationUri='s3a://incubation-workshops-bucket/hk_aug_schema0')]

In [9]:
# test_query = f"""select * from {catalog_name}.{schema_name}.{cards_silver_table} limit 10"""
# spark.sql(test_query).show(truncate=False)

# Customer risk 360

## Create customer risk 360 table

In [10]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Load the three tables
customers_df = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.{customers_silver_table}")
cards_df = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.{cards_silver_table}")
transactions_df = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.{transactions_silver_table}")

# Convert transaction_date to proper timestamp and extract year-month
transactions_df = transactions_df.withColumn(
    "transaction_timestamp", 
    to_timestamp(col("transaction_date"), "yyyy-MM-dd HH:mm:ss")
).withColumn(
    "year_month", 
    date_format(col("transaction_timestamp"), "yyyy-MM")
)

# Convert date columns in other tables
customers_df = customers_df.withColumn("dob", to_date(col("dob"), "yyyy-MM-dd")) \
                          .withColumn("acquisition_date", to_date(col("acquisition_date"), "yyyy-MM-dd"))

cards_df = cards_df.withColumn("issue_date", to_date(col("issue_date"), "yyyy-MM-dd"))

print("=== CUSTOMER MONTHLY RISK 360 ANALYSIS ===")

# 1. TRANSACTION BEHAVIOR RISK METRICS
print("\n1. Creating Transaction Behavior Risk Metrics...")

transaction_risk_metrics = transactions_df.groupBy("customer_id", "year_month").agg(
    # Volume metrics
    count("transaction_id").alias("total_transactions"),
    sum("amount_usd").alias("total_spend_usd"),
    avg("amount_usd").alias("avg_transaction_amount"),
    max("amount_usd").alias("max_transaction_amount"),
    stddev("amount_usd").alias("spend_volatility"),
    
    # International activity
    sum(when(col("is_international") == True, 1).otherwise(0)).alias("international_transactions"),
    sum(when(col("is_international") == True, col("amount_usd")).otherwise(0)).alias("international_spend"),
    
    # Payment method diversity
    countDistinct("payment_method").alias("payment_methods_used"),
    countDistinct("merchant_country").alias("countries_transacted"),
    countDistinct("category").alias("spending_categories"),
    
    # Failed transactions
    sum(when(col("status") == "Pending", 1).otherwise(0)).alias("pending_transactions"),
    sum(when(col("status") != "Approved", 1).otherwise(0)).alias("failed_transactions"),
    
    # ATM usage
    sum(when(col("atm_id").isNotNull(), 1).otherwise(0)).alias("atm_transactions"),
    sum(when(col("atm_id").isNotNull(), col("amount_usd")).otherwise(0)).alias("atm_spend"),
    
    # High-risk categories (assuming these are riskier)
    sum(when(col("category").isin("Entertainment", "Travel"), col("amount_usd")).otherwise(0)).alias("high_risk_category_spend"),
    
    # Fees and rewards
    sum("processing_fee").alias("total_fees_paid"),
    sum("reward_points").alias("total_rewards_earned")
)

# 2. CREDIT UTILIZATION RISK
print("\n2. Creating Credit Utilization Risk Metrics...")

# Get customer's total credit limit across all cards
customer_credit_limits = cards_df.groupBy("customer_id").agg(
    sum("credit_limit").alias("total_credit_limit"),
    count("card_id").alias("total_cards"),
    avg("credit_limit").alias("avg_credit_limit"),
    max("credit_limit").alias("max_credit_limit")
)

# Calculate utilization ratio
utilization_risk = transaction_risk_metrics.join(
    customer_credit_limits, "customer_id", "left"
).withColumn(
    "credit_utilization_ratio",
    col("total_spend_usd") / col("total_credit_limit")
).withColumn(
    "spend_per_card",
    col("total_spend_usd") / col("total_cards")
)

# 3. CUSTOMER PROFILE RISK FACTORS
print("\n3. Adding Customer Profile Risk Factors...")

# Calculate customer age and tenure
current_date = current_date()
customer_profile_risk = customers_df.withColumn(
    "customer_age_years",
    floor(datediff(current_date, col("dob")) / 365.25)
).withColumn(
    "customer_tenure_years",
    floor(datediff(current_date, col("acquisition_date")) / 365.25)
).withColumn(
    "loyalty_risk_score",
    when(col("loyalty_status") == "Bronze", 4)
    .when(col("loyalty_status") == "Silver", 3)
    .when(col("loyalty_status") == "Gold", 2)
    .when(col("loyalty_status") == "Platinum", 1)
    .otherwise(5)
)

# 4. TIME-BASED RISK PATTERNS
print("\n4. Creating Time-based Risk Patterns...")

# Window function for trend analysis
customer_window = Window.partitionBy("customer_id").orderBy("year_month")

# Calculate month-over-month changes
monthly_trends = utilization_risk.withColumn(
    "prev_month_spend",
    lag("total_spend_usd", 1).over(customer_window)
).withColumn(
    "prev_month_transactions",
    lag("total_transactions", 1).over(customer_window)
).withColumn(
    "spend_growth_rate",
    when(col("prev_month_spend") > 0, 
         (col("total_spend_usd") - col("prev_month_spend")) / col("prev_month_spend")
    ).otherwise(0)
).withColumn(
    "transaction_growth_rate",
    when(col("prev_month_transactions") > 0,
         (col("total_transactions") - col("prev_month_transactions")) / col("prev_month_transactions")
    ).otherwise(0)
)

# 5. COMPREHENSIVE RISK SCORING
print("\n5. Creating Comprehensive Risk Score...")

risk_360 = monthly_trends.join(
    customer_profile_risk.select("customer_id", "name", "loyalty_status", "customer_age_years", 
                                "customer_tenure_years", "loyalty_risk_score"),
    "customer_id", "left"
).withColumn(
    # Spending behavior risk (0-10 scale)
    "spending_risk_score",
    least(lit(10), 
          when(col("credit_utilization_ratio") > 0.8, 8)
          .when(col("credit_utilization_ratio") > 0.6, 6)
          .when(col("credit_utilization_ratio") > 0.4, 4)
          .when(col("credit_utilization_ratio") > 0.2, 2)
          .otherwise(1) +
          when(col("spend_volatility") > col("avg_transaction_amount") * 2, 3)
          .when(col("spend_volatility") > col("avg_transaction_amount"), 2)
          .otherwise(0)
    )
).withColumn(
    # International activity risk (0-10 scale)
    "international_risk_score",
    least(lit(10),
          when(col("international_transactions") > col("total_transactions") * 0.7, 8)
          .when(col("international_transactions") > col("total_transactions") * 0.5, 6)
          .when(col("international_transactions") > col("total_transactions") * 0.3, 4)
          .when(col("international_transactions") > 0, 2)
          .otherwise(0) +
          when(col("countries_transacted") > 5, 2)
          .when(col("countries_transacted") > 3, 1)
          .otherwise(0)
    )
).withColumn(
    # Transaction pattern risk (0-10 scale)
    "pattern_risk_score",
    least(lit(10),
          when(col("failed_transactions") > col("total_transactions") * 0.2, 6)
          .when(col("failed_transactions") > col("total_transactions") * 0.1, 4)
          .when(col("failed_transactions") > 0, 2)
          .otherwise(0) +
          when(abs(col("spend_growth_rate")) > 2.0, 4)
          .when(abs(col("spend_growth_rate")) > 1.0, 2)
          .otherwise(0)
    )
).withColumn(
    # Overall composite risk score (0-100 scale)
    "composite_risk_score",
    round(
        (col("spending_risk_score") * 0.4 + 
         col("international_risk_score") * 0.3 + 
         col("pattern_risk_score") * 0.2 + 
         col("loyalty_risk_score") * 0.1) * 10, 2
    )
).withColumn(
    "risk_category",
    when(col("composite_risk_score") >= 70, "High Risk")
    .when(col("composite_risk_score") >= 40, "Medium Risk")
    .otherwise("Low Risk")
)

# 6. FINAL CUSTOMER RISK 360 VIEW
print("\n6. Creating Final Customer Risk 360 View...")

customer_risk_360 = risk_360.select(
    col("customer_id"),
    col("name").alias("customer_name"),
    col("year_month"),
    col("loyalty_status"),
    col("customer_age_years"),
    col("customer_tenure_years"),
    
    # Transaction metrics
    col("total_transactions"),
    col("total_spend_usd"),
    col("avg_transaction_amount"),
    col("spend_volatility"),
    
    # Risk indicators
    col("credit_utilization_ratio"),
    col("international_transactions"),
    col("failed_transactions"),
    col("countries_transacted"),
    col("payment_methods_used"),
    
    # Growth patterns
    col("spend_growth_rate"),
    col("transaction_growth_rate"),
    
    # Risk scores
    col("spending_risk_score"),
    col("international_risk_score"),
    col("pattern_risk_score"),
    col("composite_risk_score"),
    col("risk_category")
).orderBy("composite_risk_score", ascending=False)

# Display results
print("\n=== TOP 10 HIGHEST RISK CUSTOMERS ===")
customer_risk_360.show(10, truncate=False)

print("\n=== RISK DISTRIBUTION ===")
customer_risk_360.groupBy("risk_category").count().show()

print("\n=== MONTHLY RISK SUMMARY ===")
customer_risk_360.groupBy("year_month").agg(
    count("customer_id").alias("total_customers"),
    avg("composite_risk_score").alias("avg_risk_score"),
    sum(when(col("risk_category") == "High Risk", 1).otherwise(0)).alias("high_risk_customers"),
    sum(when(col("risk_category") == "Medium Risk", 1).otherwise(0)).alias("medium_risk_customers"),
    sum(when(col("risk_category") == "Low Risk", 1).otherwise(0)).alias("low_risk_customers")
).orderBy("year_month").show()


=== CUSTOMER MONTHLY RISK 360 ANALYSIS ===

1. Creating Transaction Behavior Risk Metrics...

2. Creating Credit Utilization Risk Metrics...

3. Adding Customer Profile Risk Factors...

4. Creating Time-based Risk Patterns...

5. Creating Comprehensive Risk Score...

6. Creating Final Customer Risk 360 View...

=== TOP 10 HIGHEST RISK CUSTOMERS ===
+------------------------------------+----------------------+----------+--------------+------------------+---------------------+------------------+------------------+----------------------+------------------+------------------------+--------------------------+-------------------+--------------------+--------------------+------------------+-----------------------+-------------------+------------------------+------------------+--------------------+-------------+
|customer_id                         |customer_name         |year_month|loyalty_status|customer_age_years|customer_tenure_years|total_transactions|total_spend_usd   |avg_transaction_am

## Save to Iceberg table

In [11]:
# 7. SAVE RESULTS
print("\n7. Saving Customer Risk 360 to table...")

customer_risk_360.write \
    .mode("overwrite") \
    .option("path", f"s3a://incubation-workshops-bucket/{schema_name}/{gold_risk_table}") \
    .saveAsTable(f"{catalog_name}.{schema_name}.{gold_risk_table}")

print("✅ Customer Monthly Risk 360 analysis completed!")
print(f"📊 Results saved to: {catalog_name}.{schema_name}.{gold_risk_table}")



7. Saving Customer Risk 360 to table...
✅ Customer Monthly Risk 360 analysis completed!
📊 Results saved to: hk_aug_catalog0.hk_aug_schema.xy2_customers_risk_360


# Customer campaign 360

In [12]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [13]:
# Load the three tables
customers_df = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.{customers_silver_table}")
cards_df = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.{cards_silver_table}")
transactions_df = spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.{transactions_silver_table}")

# Convert transaction_date to proper timestamp and create time periods
transactions_df = transactions_df.withColumn(
    "transaction_timestamp", 
    to_timestamp(col("transaction_date"), "yyyy-MM-dd HH:mm:ss")
).withColumn(
    "year_month", 
    date_format(col("transaction_timestamp"), "yyyy-MM")
).withColumn(
    "quarter_year",
    concat(lit("Q"), quarter(col("transaction_timestamp")), lit("-"), year(col("transaction_timestamp")))
).withColumn(
    # Create 3-month rolling periods (current approach)
    "rolling_3m_period",
    date_format(
        date_sub(last_day(col("transaction_timestamp")), 
                 ((month(col("transaction_timestamp")) - 1) % 3) * 30), 
        "yyyy-MM"
    )
)

# Convert date columns in other tables
customers_df = customers_df.withColumn("dob", to_date(col("dob"), "yyyy-MM-dd")) \
                          .withColumn("acquisition_date", to_date(col("acquisition_date"), "yyyy-MM-dd"))

print("=== CUSTOMER CAMPAIGN ANALYSIS ===")

# 1. SPENDING CHANNEL ANALYSIS (3-Month Periods)
print("\n1. Creating Spending Channel Analysis...")

channel_analysis = transactions_df.groupBy("customer_id", "rolling_3m_period").agg(
    # Total spending metrics
    sum("amount_usd").alias("total_spend_3m"),
    count("transaction_id").alias("total_transactions_3m"),
    avg("amount_usd").alias("avg_transaction_amount_3m"),
    
    # Payment method preferences (channels)
    sum(when(col("payment_method") == "Contactless", col("amount_usd")).otherwise(0)).alias("contactless_spend"),
    sum(when(col("payment_method") == "Chip", col("amount_usd")).otherwise(0)).alias("chip_spend"),
    sum(when(col("payment_method") == "Online", col("amount_usd")).otherwise(0)).alias("online_spend"),
    sum(when(col("payment_method") == "ATM", col("amount_usd")).otherwise(0)).alias("atm_spend"),
    sum(when(col("payment_method") == "Swipe", col("amount_usd")).otherwise(0)).alias("swipe_spend"),
    sum(when(col("payment_method") == "Recurring", col("amount_usd")).otherwise(0)).alias("recurring_spend"),
    
    # Channel transaction counts
    sum(when(col("payment_method") == "Contactless", 1).otherwise(0)).alias("contactless_transactions"),
    sum(when(col("payment_method") == "Online", 1).otherwise(0)).alias("online_transactions"),
    sum(when(col("payment_method") == "Recurring", 1).otherwise(0)).alias("recurring_transactions"),
    
    # Geographic preferences
    sum(when(col("is_international") == True, col("amount_usd")).otherwise(0)).alias("international_spend_3m"),
    sum(when(col("is_international") == False, col("amount_usd")).otherwise(0)).alias("domestic_spend_3m"),
    countDistinct("merchant_country").alias("countries_shopped_3m")
)

# 2. CATEGORY SPENDING ANALYSIS
print("\n2. Creating Category Spending Analysis...")

category_analysis = transactions_df.groupBy("customer_id", "rolling_3m_period").agg(
    # Specific categories
    sum(when(col("category") == "Travel", col("amount_usd")).otherwise(0)).alias("travel_spend"),
    sum(when(col("category") == "Healthcare", col("amount_usd")).otherwise(0)).alias("healthcare_spend"),
    sum(when(col("category") == "Shopping", col("amount_usd")).otherwise(0)).alias("shopping_spend"),
    sum(when(col("category") == "Electronics", col("amount_usd")).otherwise(0)).alias("electronics_spend"),
    sum(when(col("category") == "Gas", col("amount_usd")).otherwise(0)).alias("gas_spend"),
    sum(when(col("category") == "Utilities", col("amount_usd")).otherwise(0)).alias("utilities_spend"),
    sum(when(col("category") == "Entertainment", col("amount_usd")).otherwise(0)).alias("entertainment_spend"),
    
    # Category transaction counts
    sum(when(col("category") == "Travel", 1).otherwise(0)).alias("travel_transactions"),
    sum(when(col("category") == "Healthcare", 1).otherwise(0)).alias("healthcare_transactions"),
    sum(when(col("category") == "Shopping", 1).otherwise(0)).alias("shopping_transactions"),
    sum(when(col("category") == "Electronics", 1).otherwise(0)).alias("electronics_transactions"),
    
    # Category diversity
    countDistinct("category").alias("spending_categories_3m"),
    
    # Lifestyle indicators
    sum(when(col("category").isin("Travel", "Entertainment"), col("amount_usd")).otherwise(0)).alias("lifestyle_spend"),
    sum(when(col("category").isin("Healthcare", "Utilities"), col("amount_usd")).otherwise(0)).alias("essential_spend"),
    sum(when(col("category").isin("Shopping", "Electronics"), col("amount_usd")).otherwise(0)).alias("retail_spend")
)

# 3. CUSTOMER PROFILE ENRICHMENT
print("\n3. Adding Customer Profile Data...")

current_date = current_date()

customer_profile = customers_df.withColumn(
    "customer_age_years",
    floor(datediff(current_date, col("dob")) / 365.25)
).withColumn(
    "customer_tenure_years",
    floor(datediff(current_date, col("acquisition_date")) / 365.25)
).withColumn(
    "age_segment",
    when(col("customer_age_years") < 25, "Gen Z (18-24)")
    .when(col("customer_age_years") < 35, "Millennial (25-34)")
    .when(col("customer_age_years") < 45, "Gen X (35-44)")
    .when(col("customer_age_years") < 55, "Boomer (45-54)")
    .otherwise("Senior (55+)")
).withColumn(
    "tenure_segment",
    when(col("customer_tenure_years") < 1, "New (< 1 year)")
    .when(col("customer_tenure_years") < 3, "Growing (1-3 years)")
    .when(col("customer_tenure_years") < 7, "Established (3-7 years)")
    .otherwise("Veteran (7+ years)")
)

# 4. CAMPAIGN TARGETING SEGMENTS
print("\n4. Creating Campaign Targeting Segments...")

# Combine all analyses
campaign_base = channel_analysis.join(category_analysis, ["customer_id", "rolling_3m_period"], "inner") \
                               .join(customer_profile.select("customer_id", "name", "loyalty_status", 
                                                           "customer_age_years", "customer_tenure_years", 
                                                           "age_segment", "tenure_segment"),
                                    "customer_id", "left")

# Calculate spending behavior percentages and preferences
campaign_segments = campaign_base.withColumn(
    # Channel preferences (as percentages)
    "contactless_pct", round((col("contactless_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "online_pct", round((col("online_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "recurring_pct", round((col("recurring_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "international_pct", round((col("international_spend_3m") / col("total_spend_3m")) * 100, 2)
).withColumn(
    # Category preferences (as percentages)
    "travel_pct", round((col("travel_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "healthcare_pct", round((col("healthcare_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "lifestyle_pct", round((col("lifestyle_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "essential_pct", round((col("essential_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    "retail_pct", round((col("retail_spend") / col("total_spend_3m")) * 100, 2)
).withColumn(
    # Primary channel identification
    "primary_channel",
    when(col("online_pct") >= 40, "Digital Native")
    .when(col("contactless_pct") >= 40, "Contactless Preferred")
    .when(col("recurring_pct") >= 30, "Subscription Focused")
    .when(col("atm_spend") > col("total_spend_3m") * 0.3, "Cash Heavy")
    .otherwise("Traditional Spender")
).withColumn(
    # Primary spending category
    "primary_category",
    when(col("travel_pct") >= 30, "Travel Enthusiast")
    .when(col("healthcare_pct") >= 25, "Health Conscious")
    .when(col("retail_pct") >= 40, "Retail Shopper")
    .when(col("lifestyle_pct") >= 35, "Lifestyle Spender")
    .when(col("essential_pct") >= 50, "Essential Spender")
    .otherwise("Balanced Spender")
).withColumn(
    # Spending tier
    "spending_tier",
    when(col("total_spend_3m") >= 5000, "High Spender")
    .when(col("total_spend_3m") >= 2000, "Medium Spender")
    .when(col("total_spend_3m") >= 500, "Low Spender")
    .otherwise("Minimal Spender")
).withColumn(
    # Campaign targeting score (0-100)
    "campaign_attractiveness_score",
    round(
        (when(col("total_spend_3m") >= 2000, 25).otherwise(col("total_spend_3m") / 80)) +
        (when(col("loyalty_status") == "Platinum", 25)
         .when(col("loyalty_status") == "Gold", 20)
         .when(col("loyalty_status") == "Silver", 15)
         .otherwise(10)) +
        (when(col("spending_categories_3m") >= 5, 20)
         .when(col("spending_categories_3m") >= 3, 15)
         .otherwise(10)) +
        (when(col("international_pct") >= 20, 15)
         .when(col("countries_shopped_3m") >= 3, 10)
         .otherwise(5)) +
        (when(col("customer_tenure_years") >= 3, 15)
         .when(col("customer_tenure_years") >= 1, 10)
         .otherwise(5)), 2
    )
)

# 5. CAMPAIGN RECOMMENDATIONS
print("\n5. Creating Campaign Recommendations...")

campaign_recommendations = campaign_segments.withColumn(
    "recommended_campaigns",
    concat_ws(", ",
        when(col("travel_pct") >= 20, "Travel Rewards"),
        when(col("healthcare_pct") >= 15, "Health & Wellness"),
        when(col("online_pct") >= 30, "E-commerce Cashback"),
        when(col("international_pct") >= 15, "International Benefits"),
        when(col("contactless_pct") >= 40, "Contactless Promotions"),
        when(col("lifestyle_pct") >= 25, "Lifestyle Perks"),
        when(col("spending_tier") == "High Spender", "Premium Services"),
        when(col("age_segment").contains("Gen Z"), "Digital Engagement"),
        when(col("age_segment").contains("Senior"), "Simplified Services")
    )
).withColumn(
    "campaign_priority",
    when(col("campaign_attractiveness_score") >= 80, "High Priority")
    .when(col("campaign_attractiveness_score") >= 60, "Medium Priority")
    .when(col("campaign_attractiveness_score") >= 40, "Low Priority")
    .otherwise("Monitor Only")
)

# 6. FINAL CAMPAIGN ANALYSIS TABLE
print("\n6. Creating Final Campaign Analysis View...")

customer_campaign_360 = campaign_recommendations.select(
    col("customer_id"),
    col("name").alias("customer_name"),
    col("rolling_3m_period"),
    col("age_segment"),
    col("tenure_segment"),
    col("loyalty_status"),
    
    # Spending overview
    col("total_spend_3m"),
    col("total_transactions_3m"),
    col("avg_transaction_amount_3m"),
    col("spending_tier"),
    
    # Channel preferences
    col("primary_channel"),
    col("online_pct"),
    col("contactless_pct"),
    col("recurring_pct"),
    col("international_pct"),
    
    # Category preferences  
    col("primary_category"),
    col("travel_pct"),
    col("healthcare_pct"),
    col("lifestyle_pct"),
    col("retail_pct"),
    col("essential_pct"),
    
    # Campaign targeting
    col("campaign_attractiveness_score"),
    col("campaign_priority"),
    col("recommended_campaigns")
).orderBy("campaign_attractiveness_score", ascending=False)

# Display results
print("\n=== TOP 10 CAMPAIGN TARGET CUSTOMERS ===")
customer_campaign_360.show(10, truncate=False)

print("\n=== CAMPAIGN PRIORITY DISTRIBUTION ===")
customer_campaign_360.groupBy("campaign_priority").count().orderBy("count", ascending=False).show()

print("\n=== PRIMARY CHANNEL DISTRIBUTION ===")
customer_campaign_360.groupBy("primary_channel").agg(
    count("customer_id").alias("customer_count"),
    avg("total_spend_3m").alias("avg_spend_3m")
).orderBy("customer_count", ascending=False).show()

print("\n=== PRIMARY CATEGORY DISTRIBUTION ===")
customer_campaign_360.groupBy("primary_category").agg(
    count("customer_id").alias("customer_count"),
    avg("campaign_attractiveness_score").alias("avg_attractiveness_score")
).orderBy("customer_count", ascending=False).show()

print("\n=== AGE SEGMENT ANALYSIS ===")
customer_campaign_360.groupBy("age_segment").agg(
    count("customer_id").alias("customer_count"),
    avg("total_spend_3m").alias("avg_spend_3m"),
    avg("campaign_attractiveness_score").alias("avg_attractiveness_score")
).orderBy("avg_spend_3m", ascending=False).show()

=== CUSTOMER CAMPAIGN ANALYSIS ===

1. Creating Spending Channel Analysis...

2. Creating Category Spending Analysis...

3. Adding Customer Profile Data...

4. Creating Campaign Targeting Segments...

5. Creating Campaign Recommendations...

6. Creating Final Campaign Analysis View...

=== TOP 10 CAMPAIGN TARGET CUSTOMERS ===
+------------------------------------+--------------------+-----------------+------------------+-----------------------+--------------+------------------+---------------------+-------------------------+--------------+---------------------+----------+---------------+-------------+-----------------+-----------------+----------+--------------+-------------+----------+-------------+-----------------------------+-----------------+----------------------------------------------------------------------------------------------------------------------+
|customer_id                         |customer_name       |rolling_3m_period|age_segment       |tenure_segment         |loy

In [14]:
# 7. SAVE RESULTS
print("\n7. Saving Customer Campaign Analysis to table...")

customer_campaign_360.write \
    .mode("overwrite") \
    .option("path", f"s3a://incubation-workshops-bucket/{schema_name}/{gold_campaign_table}") \
    .saveAsTable(f"{catalog_name}.{schema_name}.{gold_campaign_table}")

print("✅ Customer Campaign Analysis completed!")
print(f"📊 Results saved to: {catalog_name}.{schema_name}.{gold_campaign_table}")


7. Saving Customer Campaign Analysis to table...
✅ Customer Campaign Analysis completed!
📊 Results saved to: hk_aug_catalog0.hk_aug_schema.xy2_customers_campaign_360
